In [ ]:
import copy
import numpy as np
import open3d as o3
from probreg import cpd
import transforms3d as t3d

def estimate_normals(pcd, params):
    pcd.estimate_normals(search_param=params)
    pcd.orient_normals_to_align_with_direction()


def prepare_source_and_target_rigid_3d(source_filename,
                                       noise_amp=0.001,
                                       n_random=500,
                                       orientation=np.deg2rad([0.0, 0.0, 30.0]),
                                       translation=np.zeros(3),
                                       voxel_size=0.005,
                                       normals=False):
    source = o3.io.read_point_cloud(source_filename)
    source = source.voxel_down_sample(voxel_size=voxel_size)
    print(source)
    target = copy.deepcopy(source)
    tp = np.asarray(target.points)
    np.random.shuffle(tp)
    rg = 1.5 * (tp.max(axis=0) - tp.min(axis=0))
    rands = (np.random.rand(n_random, 3) - 0.5) * rg + tp.mean(axis=0)
    target.points = o3.utility.Vector3dVector(np.r_[tp + noise_amp * np.random.randn(*tp.shape), rands])
    ans = np.identity(4)
    ans[:3, :3] = t3d.euler.euler2mat(*orientation)
    ans[:3, 3] = translation
    target.transform(ans)
    if normals:
        estimate_normals(source, o3.geometry.KDTreeSearchParamHybrid(radius=0.3, max_nn=50))
        estimate_normals(target, o3.geometry.KDTreeSearchParamHybrid(radius=0.3, max_nn=50))
    return source, target

def prepare_source_and_target_nonrigid_2d(source_filename,
                                          target_filename):
    source = np.loadtxt(source_filename)
    target = np.loadtxt(target_filename)
    return source, target


def prepare_source_and_target_nonrigid_3d(source_filename,
                                          target_filename,
                                          voxel_size=5.0):
    source = o3.geometry.PointCloud()
    target = o3.geometry.PointCloud()
    source.points = o3.utility.Vector3dVector(np.loadtxt(source_filename))
    target.points = o3.utility.Vector3dVector(np.loadtxt(target_filename))
    source = source.voxel_down_sample(voxel_size=voxel_size)
    target = target.voxel_down_sample(voxel_size=voxel_size)
    print(source)
    print(target)
    return source, target

# ICP

In [ ]:
A = np.random.randint(0,101,(200,3))  # 20 points for test
rotz = lambda theta: np.array([[np.cos(theta),-np.sin(theta),0],
                                   [np.sin(theta),np.cos(theta),0],
                                   [0,0,1]])
trans = np.array([2.12,-0.2,1.3])
B = A.dot(rotz(np.pi/4).T) + trans 


# Pass xyz to Open3D.o3d.geometry.PointCloud and visualize
# pcd = o3.geometry.PointCloud()
target.points = o3.utility.Vector3dVector(A)
source.points = o3.utility.Vector3dVector(B)
# source, target = prepare_source_and_target_rigid_3d('G:/probreg/examples/bunny.pcd')
# target = copy.deepcopy(source)

# vis = o3.visualization.Visualizer()
# vis.create_window()
result = copy.deepcopy(source)
source.paint_uniform_color([1, 0, 0])
target.paint_uniform_color([0, 1, 0])
result.paint_uniform_color([0, 0, 1])
# vis.add_geometry(source)
# vis.add_geometry(target)
# vis.add_geometry(result)
threshold = 0.05 # how to set initially
icp_iteration = 100
save_image = False

for i in range(icp_iteration):
    reg_p2p = o3.pipelines.registration.registration_icp(result, target, threshold,
                np.identity(4), o3.pipelines.registration.TransformationEstimationPointToPoint(),
                o3.pipelines.registration.ICPConvergenceCriteria(max_iteration=1))
    result.transform(reg_p2p.transformation)
#     vis.update_geometry(source)
#     vis.update_geometry(target)
#     vis.update_geometry(result)
#     vis.poll_events()
#     vis.update_renderer()
#     if save_image:
#         vis.capture_screen_image("image_%04d.jpg" % i)
# vis.run()
o3.visualization.draw_geometries([result,source,target])

# CPD

In [5]:
import copy
import numpy as np
import open3d as o3
from probreg import cpd
import transforms3d as t3d

def estimate_normals(pcd, params):
    pcd.estimate_normals(search_param=params)
    pcd.orient_normals_to_align_with_direction()# load source and target point cloud
    
source = o3.io.read_point_cloud('D:/0_software/github/probreg/examples/bunny.pcd')
source.remove_non_finite_points()
target = copy.deepcopy(source)
# transform target point cloud
th = np.deg2rad(30.0)
target.transform(np.array([[np.cos(th), -np.sin(th), 0.0, 0.0],
                           [np.sin(th), np.cos(th), 0.0, 0.0],
                           [0.0, 0.0, 1.0, 0.0],
                           [0.0, 0.0, 0.0, 1.0]]))
source = source.voxel_down_sample(voxel_size=0.005)
target = target.voxel_down_sample(voxel_size=0.005)

# compute cpd registration
tf_param, _, _ = cpd.registration_cpd(source, target)
result = copy.deepcopy(source)
result.points = tf_param.transform(result.points)

# draw result
source.paint_uniform_color([1, 0, 0])
target.paint_uniform_color([0, 1, 0])
result.paint_uniform_color([0, 0.5, 1])
o3.visualization.draw_geometries([source, target, result])

# CPD with point cloud

In [4]:
import copy
import numpy as np
import open3d as o3
from probreg import cpd
import transforms3d as t3d

def estimate_normals(pcd, params):
    pcd.estimate_normals(search_param=params)
    pcd.orient_normals_to_align_with_direction()# load source and target point cloud

# read points of # point cloud
# xyz = np.asarray(result.points)

A = np.random.randint(0,101,(200,3))  # 20 points for test
rotz = lambda theta: np.array([[np.cos(theta),-np.sin(theta),0],
                                   [np.sin(theta),np.cos(theta),0],
                                   [0,0,1]])
trans = np.array([2.12,-0.2,1.3])
B = A.dot(rotz(np.pi/4).T) + trans 

# Pass xyz to Open3D.o3d.geometry.PointCloud and visualize
# pcd = o3.geometry.PointCloud()
target.points = o3.utility.Vector3dVector(A)
source.points = o3.utility.Vector3dVector(B)
# o3.io.write_point_cloud("../../test_data/sync.ply", pcd)
# o3.visualization.draw_geometries(pcd)

# register
tf_param, _, _ = cpd.registration_cpd(source, target)
result = copy.deepcopy(source)
result.points = tf_param.transform(result.points)

# draw result
source.paint_uniform_color([1, 0, 0])
target.paint_uniform_color([0, 1, 0])
result.paint_uniform_color([0, 0.5, 1])
o3.visualization.draw_geometries([result,source])

In [ ]:
# print(np.asarray(source.points).shape)
# print(np.asarray(target.points).shape)
# print(np.asarray(result.points).shape)
# o3.visualization.draw_geometries([result])

# o3.io.write_point_cloud("../../test_data/sync.ply", pcd) # Function to write PointCloud to file

# cpd_rigid_cuda

In [ ]:
import numpy as np
use_cuda = True
if use_cuda:
    import cupy as cp
    to_cpu = cp.asnumpy
    cp.cuda.set_allocator(cp.cuda.MemoryPool().malloc)
else:
    cp = np
    to_cpu = lambda x: x
import open3d as o3
import transforms3d as trans
from probreg import cpd
from probreg import callbacks
import utils
import time

source, target = utils.prepare_source_and_target_rigid_3d('bunny.pcd', voxel_size=0.003)
source = cp.asarray(source.points, dtype=cp.float32)
target = cp.asarray(target.points, dtype=cp.float32)

rcpd = cpd.RigidCPD(source, use_cuda=use_cuda)
start = time.time()
tf_param, _, _ = rcpd.registration(target)
elapsed = time.time() - start
print("time: ", elapsed)

print("result: ", np.rad2deg(t3d.euler.mat2euler(to_cpu(tf_param.rot))),
      tf_param.scale, to_cpu(tf_param.t))

# cpd_affine3d_cuda

In [ ]:
import numpy as np
use_cuda = True
if use_cuda:
    import cupy as cp
    to_cpu = cp.asnumpy
    cp.cuda.set_allocator(cp.cuda.MemoryPool().malloc)
else:
    cp = np
    to_cpu = lambda x: x
import open3d as o3
from probreg import cpd
from probreg import callbacks
import utils
import time

source, target = utils.prepare_source_and_target_nonrigid_3d('face-x.txt', 'face-y.txt', voxel_size=3.0)
source = cp.asarray(source.points, dtype=cp.float32)
target = cp.asarray(target.points, dtype=cp.float32)

acpd = cpd.AffineCPD(source, use_cuda=use_cuda)
start = time.time()
tf_param, _, _ = acpd.registration(target)
elapsed = time.time() - start
print("time: ", elapsed)

print("result: ", to_cpu(tf_param.b), to_cpu(tf_param.t))

# gmmtree_rigid

In [ ]:
import numpy as np
import transforms3d as t3d
from probreg import gmmtree
from probreg import callbacks
import utils

source, target = utils.prepare_source_and_target_rigid_3d('bunny.pcd', n_random=0)

cbs = [callbacks.Open3dVisualizerCallback(source, target)]
tf_param, _ = gmmtree.registration_gmmtree(source, target, callbacks=cbs)

print("result: ", np.rad2deg(t3d.euler.mat2euler(tf_param.rot)),
      tf_param.scale, tf_param.t)

# filterreg_rigid

In [ ]:
import numpy as np
import transforms3d as t3d
from probreg import filterreg
from probreg import callbacks
import utils

source, target = utils.prepare_source_and_target_rigid_3d('bunny.pcd')

cbs = [callbacks.Open3dVisualizerCallback(source, target)]
objective_type = 'pt2pt'
tf_param, _, _ = filterreg.registration_filterreg(source, target,
                                                  objective_type=objective_type,
                                                  sigma2=None,
                                                  update_sigma2=True,
                                                  callbacks=cbs)

print("result: ", np.rad2deg(t3d.euler.mat2euler(tf_param.rot)),
      tf_param.scale, tf_param.t)

# svr_rigid

In [1]:
import numpy as np
import transforms3d as t3d
from probreg import l2dist_regs
from probreg import callbacks
# import utils

source, target = utils.prepare_source_and_target_rigid_3d('bunny.pcd', n_random=0)

cbs = [callbacks.Open3dVisualizerCallback(source, target)]
tf_param = l2dist_regs.registration_svr(source, target, callbacks=cbs)

print("result: ", np.rad2deg(t3d.euler.mat2euler(tf_param.rot)),
      tf_param.scale, tf_param.t)

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


ModuleNotFoundError: No module named 'utils'

# cpd_nonrigid3d_cuda.py

In [8]:
import numpy as np
use_cuda = True
if use_cuda:
    import cupy as cp
    to_cpu = cp.asnumpy
    cp.cuda.set_allocator(cp.cuda.MemoryPool().malloc)
else:
    cp = np
    to_cpu = lambda x: x
import open3d as o3
from probreg import cpd
from probreg import callbacks
# import utils
import time

# source, target = prepare_source_and_target_nonrigid_3d('D:/0_software/github/probreg/examples/face-x.txt', 'D:/0_software/github/probreg/examples/face-y.txt', voxel_size=5.0)
source_pt = cp.asarray(source.points, dtype=cp.float32)
target_pt = cp.asarray(target.points, dtype=cp.float32)

acpd = cpd.NonRigidCPD(source_pt, use_cuda=use_cuda)
start = time.time()
tf_param, _, _ = acpd.registration(target_pt)
elapsed = time.time() - start
print("time: ", elapsed)

print("result: ", to_cpu(tf_param.w), to_cpu(tf_param.g))

result = tf_param.transform(source_pt)
pc = o3.geometry.PointCloud()
pc.points = o3.utility.Vector3dVector(to_cpu(result))
pc.paint_uniform_color([0, 1, 0])
target.paint_uniform_color([0, 0, 1])
o3.visualization.draw_geometries([pc, source,target])

time:  0.2964601516723633
result:  [[ 2.05505882 -0.92490157  0.93029277]
 [-0.79028673  0.48508255 -0.96037097]
 [-0.66808533  1.14580345 -0.69689935]
 ...
 [ 1.82050339 -0.58081339 -2.82333118]
 [-1.9659839  -0.36522374  0.35111367]
 [-1.92147476 -0.44572055  0.51062099]] [[1.         0.9981962  0.99893767 ... 0.9954284  0.99813515 0.99811935]
 [0.9981962  1.         0.99935746 ... 0.99111515 0.9936516  0.9936    ]
 [0.99893767 0.99935746 1.         ... 0.9915706  0.99477583 0.9947912 ]
 ...
 [0.9954284  0.99111515 0.9915706  ... 1.         0.99900573 0.99899477]
 [0.99813515 0.9936516  0.99477583 ... 0.99900573 1.         0.9999975 ]
 [0.99811935 0.9936     0.9947912  ... 0.99899477 0.9999975  1.        ]]


# bcpd_nonrigid.py 

In [9]:
import numpy as np
import open3d as o3
import transforms3d as t3d
from probreg import bcpd
from probreg import callbacks
# import utils

# source, target = prepare_source_and_target_nonrigid_3d('D:/0_software/github/probreg/examples/bunny-x.txt','D:/0_software/github/probreg/examples/bunny-y.txt', 0.1)
cbs = [callbacks.Open3dVisualizerCallback(source, target)]
tf_param = bcpd.registration_bcpd(source, target,callbacks=cbs)

print("result: ", np.rad2deg(t3d.euler.mat2euler(tf_param.rigid_trans.rot)),tf_param.rigid_trans.scale, tf_param.rigid_trans.t)

result:  [-0.01578165  0.01471676  0.00152319] 0.33544550420297714 [-0.04948553  0.0496432   0.01958222]


# filterreg_deformable

In [ ]:
import copy
import numpy as np
import open3d as o3
from probreg import filterreg
from probreg import callbacks
from probreg import transformation
from dq3d import dualquat, quat

n_points = 30
points = np.array([[i * 0.05, 0.0, 0.0] for i in range(n_points)])
tfs = [(quat(np.deg2rad(0.0), np.array([0.0, 0.0, 1.0])), np.array([0.0, 0.0, 0.0])),
       (quat(np.deg2rad(30.0), np.array([0.0, 0.0, 1.0])), np.array([0.0, 0.0, 0.3]))]
dqs = [dualquat(t[0], t[1]) for t in tfs]
ws = transformation.DeformableKinematicModel.SkinningWeight(n_points)
for i in range(n_points):
    ws['pair'][i][0] = 0
    ws['pair'][i][1] = 1
for i in range(n_points):
    ws['val'][i][0] = float(i) / n_points
    ws['val'][i][1] = 1.0 - float(i) / n_points
dtf = transformation.DeformableKinematicModel(dqs, ws)
tf_points = dtf.transform(points)

source = o3.geometry.PointCloud()
source.points = o3.utility.Vector3dVector(points)
target = o3.geometry.PointCloud()
target.points = o3.utility.Vector3dVector(tf_points)

cbs = [callbacks.Open3dVisualizerCallback(source, target)]
cv = lambda x: np.asarray(x.points if isinstance(x, o3.geometry.PointCloud) else x)
reg = filterreg.DeformableKinematicFilterReg(cv(source), ws, 0.01)
reg.set_callbacks(cbs)
tf_param, _, _ = reg.registration(cv(target))

print(dqs)
print(tf_param.dualquats)

# time_measurement.

In [ ]:
from timeit import default_timer as timer
import numpy as np
import open3d as o3
import utils
from probreg import cpd
from probreg import l2dist_regs
from probreg import gmmtree
from probreg import filterreg

threshold = 0.001
max_iteration = 100

source, target = utils.prepare_source_and_target_rigid_3d('bunny.pcd',  n_random=0,
                                                          orientation=np.deg2rad([0.0, 0.0, 10.0]))

start = timer()
res = o3.pipelines.registration.registration_icp(source, target, 0.5,
                                                 np.identity(4), o3.pipelines.registration.TransformationEstimationPointToPoint(),
                                                 o3.pipelines.registration.ICPConvergenceCriteria(max_iteration=max_iteration))
end = timer()
print('ICP(Open3D): ', end - start)

start = timer()
res = cpd.registration_cpd(source, target, maxiter=max_iteration, tol=threshold)
end = timer()
print('CPD: ', end - start)

start = timer()
res = l2dist_regs.registration_svr(source, target, opt_maxiter=max_iteration, opt_tol=threshold)
end = timer()
print('SVR: ', end - start)

start = timer()
res = gmmtree.registration_gmmtree(source, target, maxiter=max_iteration, tol=threshold)
end = timer()
print('GMMTree: ', end - start)

start = timer()
res = filterreg.registration_filterreg(source, target,
                                       sigma2=None, maxiter=max_iteration, tol=threshold)
end = timer()
print('FilterReg: ', end - start)

# Evaluation

In [ ]:
from scipy.spatial import KDTree
target_tree = KDTree(target, leafsize=10)
rmse = math_utils.compute_rmse(source, target_tree)